In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import re
from src.data_items import add_suffixes
from selenium.webdriver.common.keys import Keys

ModuleNotFoundError: No module named 'selenium'

In [ ]:
pricedf = pd.read_csv('src/scraping/atx_listings_trulia.csv')

In [ ]:
geodf = pd.read_csv('data/Austin_addresses.csv')

In [ ]:
geodf['c_zip_code'] = geodf['c_zip_code'].astype(str).map(lambda x: x.replace('.0', ''))
geodf.head()

In [ ]:
pricedf.head()

In [ ]:
pricedf['price'] = pricedf['price'].map(lambda x: str(x).replace(',', '').replace('$',''))

In [ ]:
pricedf['sqft'] = pricedf['sqft'].map(lambda x: str(x).replace(',', '').replace(' Square Feet',''))

In [ ]:
pricedf['sqft'].fillna(0, inplace=True)

In [ ]:
pricedf['sqft'] = pricedf['sqft'].map(lambda x: 0 if x == 'nan' else x)

In [ ]:
pricedf.house_type.unique()

In [ ]:
pricedf[pricedf['house_type'] == 'Lot Size: 9496 sqft']

In [ ]:
 def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [ ]:
newpricedf = pricedf[pricedf['sqft'].map(lambda x: True if str(x).isdigit() else False)].copy()

In [ ]:
newpricedf['price'] = newpricedf['price'].astype(int)
newpricedf['sqft'] = newpricedf['sqft'].astype(int)

In [ ]:
newpricedf["price_per_sqft"] = newpricedf['price'] / newpricedf['sqft']

In [ ]:
newpricedf.drop(['bathrooms', 'bedrooms', 'url', 'house_type'], axis=1, inplace=True)

In [ ]:
newpricedf.drop_duplicates(inplace=True)

In [ ]:
newpricedf.shape

In [ ]:
newpricedf.address.is_unique

In [ ]:
address = newpricedf.address
newpricedf[address.isin(address[address.duplicated()])]

In [ ]:
newpricedf.drop(569, inplace=True)

In [ ]:
newpricedf.address.is_unique

In [ ]:
newpricedf.address = newpricedf.address.str.upper()

In [ ]:
addlist = newpricedf.address.map(lambda x: x.split(' '))

In [ ]:
addset = set(add_suffixes)

In [ ]:
def cleanup(string):
    string = string.upper().strip('.').strip(',')
    if string in addset:
        string = add_suffixes[string]
    return string

In [ ]:
addclean = [ [ cleanup(word) for word in address] for address in addlist ]

In [ ]:
def findPound(address):
    result = ''
    for word in address:
        if re.search('#', word):
            result = word
        continue
    if result == '':
        result = 'Nan'
    return result

In [ ]:
apts = [findPound(address) for address in addclean]

In [ ]:
addclean = [[word for word in address if not re.search('#', word)] for address in addclean]

In [ ]:
joinedclean = [' '.join(address) for address in addclean]

In [ ]:
joinedclean

In [ ]:
newpricedf['city_state_zip'] = newpricedf['city_state_zip'].map(lambda x: cleanup(x).replace('AUSTIN,', ' AUSTIN'))

In [ ]:
newpricedf['adj_address'] = joinedclean + newpricedf['city_state_zip']

In [ ]:
newpricedf['apts'] = apts

In [ ]:
mergedf = newpricedf.merge(geodf, how='left', left_on='adj_address', right_on='address')

In [ ]:
geodf['spaces'] = ' '

In [ ]:
geodf['adj_address'] = geodf['c_street_number'] + geodf['spaces'] + geodf['c_street'] + geodf['spaces'] + geodf['c_city'] + geodf['spaces'] + geodf['c_state'] + geodf['spaces'] + geodf['c_zip_code']

In [ ]:
geodf.head()

In [ ]:
mergedf = newpricedf.merge(geodf, how='left', on='adj_address')

In [ ]:
mergedf